In [14]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.functions import lit, substring, col
from delta.tables import DeltaTable 
import os
from dotenv import load_dotenv
import psycopg2 
import boto3
from botocore.exceptions import ClientError

In [2]:
def create_spark_session(app_name,  notebook_name, notebook_namespace, s3_access_key, s3_secret_key, s3_endpoint):
    """
    spark 세션을 creat
    """
    conf = SparkConf()
    # Spark driver, executor 설정
    conf.set("spark.submit.deployMode", "client")
    conf.set("spark.executor.instances", "1")
    conf.set("spark.executor.memory", "1G")
    conf.set("spark.driver.memory", "1G")
    conf.set("spark.executor.cores", "1")
    conf.set("spark.kubernetes.namespace", notebook_namespace)
    conf.set("spark.kubernetes.container.image", "paasup/spark:3.5.2-java17-python3.11-2")
    conf.set("spark.kubernetes.authenticate.driver.serviceAccountName", "default-editor")
    conf.set("spark.kubernetes.driver.pod.name", os.environ["HOSTNAME"])
    conf.set("spark.driver.bindAddress", "0.0.0.0")
    conf.set("spark.driver.host", notebook_name+ "-headless." + notebook_namespace + ".svc.cluster.local")
    conf.set("spark.driver.port", "51810")        
    conf.set("spark.broadcast.port", "51811")     
    conf.set("spark.blockManager.port", "51812")

    # s3, delta, postgresql 사용 시 필요한 jar 패키지 설정
    jar_list = ",".join([
    "org.apache.hadoop:hadoop-common:3.3.4",
    "org.apache.hadoop:hadoop-aws:3.3.4",
    "com.amazonaws:aws-java-sdk:1.11.655", 
    "io.delta:delta-spark_2.12:3.3.1", # Keep if you might use Delta Lake for other operations
    "org.postgresql:postgresql:42.7.2" # Added for PostgreSQL
    ])
    conf.set("spark.jars.packages", jar_list)  

    # s3 세팅
    conf.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    conf.set('spark.hadoop.fs.s3a.path.style.access', 'true')
    conf.set('spark.hadoop.fs.s3a.connection.ssl.enabled', 'true')
    conf.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    conf.set('spark.hadoop.fs.s3a.access.key', s3_access_key)
    conf.set('spark.hadoop.fs.s3a.secret.key', s3_secret_key)
    conf.set('spark.hadoop.fs.s3a.endpoint', s3_endpoint)
    
    ### ssl 검증 비활성화
    conf.set("spark.driver.extraJavaOptions", "-Dcom.amazonaws.sdk.disableCertChecking=true")
    conf.set("spark.executor.extraJavaOptions", "-Dcom.amazonaws.sdk.disableCertChecking=true")

    # deltalake 세팅
    conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    conf.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 

    # --- SparkSession 빌드 ---
    spark = SparkSession.builder \
        .appName(app_name) \
        .config(conf=conf) \
        .getOrCreate()

    return spark

In [3]:
def move_s3_file(s3_config, source_key, dest_key):
    """
    boto3를 사용하여 S3 내에서 파일을 이동(복사 후 삭제)합니다.
    """
    s3_client = boto3.client(
        's3',
        aws_access_key_id=s3_config['access_key'],
        aws_secret_access_key=s3_config['secret_key'],
        endpoint_url=s3_config['endpoint'],
        verify=False # 자체 서명 인증서 사용 시 필요할 수 있음
    )
    try:
        copy_source = {'Bucket': s3_config['bucket'], 'Key': source_key}
        print(f"Copying S3 object from {source_key} to {dest_key}...")
        s3_client.copy_object(CopySource=copy_source, Bucket=s3_config['bucket'], Key=dest_key)
        print(f"Deleting source S3 object: {source_key}...")
        s3_client.delete_object(Bucket=s3_config['bucket'], Key=source_key)
        print(f"Successfully moved {source_key} to {dest_key}.")
        return True
    except ClientError as e:
        print(f"Error moving S3 file: {e}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred during S3 move: {e}")
        return False

In [12]:
def subway_info_overwrite(spark, s3_config, pg_config): 
    """
    CSV--> DELTA --> POSTGRESQL 처리하는 메인 파이프라인
    """
    pg_url = f"jdbc:postgresql://{pg_config['host']}:{pg_config['port']}/{pg_config['dbname']}"
    pg_properties = {
        "user": pg_config['user'],
        "password": pg_config['password'],
        "driver": "org.postgresql.Driver"
    }

    print(f" bucket name is {s3_config['bucket']}")
    csv_source_key = f"datasource/서울교통공사_노선별 지하철역 정보.csv"
    csv_archive_key = f"archive/서울교통공사_노선별 지하철역 정보.csv"
    csv_s3_path = f"s3a://{s3_config['bucket']}/{csv_source_key}"
    delta_s3_path = f"s3a://{s3_config['bucket']}/{s3_config['delta_path']}"
    
    try:    
        print(f"\n--- Starting processing for {pg_config['table']} ---")

        # --- 단계 1: S3 CSV 읽기 ---
        print(f"Reading CSV from {csv_s3_path}...")
        # CSV 스키마나 옵션(overwrite, inferSchema...)은 소스 파일에 맞게 조정
        csv_df = spark.read.option("header", "true").option("overwriteSchema", "true").csv(csv_s3_path)
        csv_df.createOrReplaceTempView("subway_info")
        csv_df.show(5)
        
        print(f"CSV  loaded. Count: {csv_df.count()}")

        # --- 단계 2: Delta Lake에 저장 (Spark SQL 사용) ---
        csv_df.write.format("delta").mode("overwrite").save(delta_s3_path)
        print(f"Successfully wrote data to Delta table {delta_s3_path}.")
        
        # --- 단계 3: S3 CSV 파일 이동 ---
        print(f"Archiving S3 CSV file...")
        move_s3_file(s3_config, csv_source_key, csv_archive_key)       

        # --- 단계 4: PostgreSQL에 삭제 후 삽입 ---   
        pg_table_name = pg_config['table']

        print(f"Inserting data  into PostgreSQL table: {pg_table_name}")
         # 데이터를 가공할 Spark SQL 쿼리
        insert_sql = f"""
                        select  `전철역명` station_name
                           ,`전철역코드` station_cd
                           ,`전철명명_영문`  station_ename
                           ,`전철명명_중문`  station_cname
                           ,`전철명명_일문`  station_jname
                           ,`외부코드`  station_cd_external
                           , case when substring(`호선`,1,1) = '0' then substring(`호선`,2,length(`호선`)-1) 
                                  when `호선` = '공항철도' then '공항철도 1호선' 
                                  when `호선` = '수인분당선' then '수인선'
                                  when `호선` = '신분당선' then '분당선'
                                  when `호선` = '우이신설경전철' then '우이신설선' 
                                  else `호선` end   line_no
                        from delta.`{delta_s3_path}`
                    """      
        print(insert_sql) 
        delta_df = spark.sql(insert_sql)
        delta_df.write \
            .jdbc(url=pg_url,
                  table=pg_table_name,
                  mode="overwrite",
                  properties=pg_properties)
        print(f"Successfully wrote data to PostgreSQL table {pg_table_name}.")

        print("--- Processing for   completed successfully! ---")

    except Exception as e:
        print(f"An error occurred during processing : {e}")
        import traceback
        traceback.print_exc()


In [13]:
# --- 메인 실행 블록 ---
if __name__ == "__main__":
    
    # Pod 내의  .env 파일에서 환경변수 read     
    dotenv_path = './config/.env'
    load_dotenv(dotenv_path=dotenv_path)    
    
    # --- APP구성 변수 설정  ---
    APP_NAME = 'load_subway_info'
    NOTEBOOK_NAME      = "test"
    NOTEBOOK_NAMESPACE = "demo01-kf"    
    S3_CONFIG = {
        "access_key": os.getenv("S3_ACCESS_KEY"),
        "secret_key": os.getenv("S3_SECRET_KEY"),
        "endpoint": os.getenv("S3_ENDPOINT_URL"),
        "bucket":  os.getenv("S3_BUCKET_NAME"),
        "delta_path": "deltalake/subway_info" # S3 버킷 내 Delta target 테이블 경로
    }

    PG_CONFIG = {
        "host": os.getenv("PG_HOST"),
        "port": os.getenv("PG_PORT", "5432"), # 기본값 5432 지정
        "dbname": os.getenv("PG_DB"),
        "user": os.getenv("PG_USER"),
        "password": os.getenv("PG_PASSWORD"),
        "table": "subway_info" # PostgreSQL target 테이블 이름
    }
    # SparkSession 생성
    spark = create_spark_session(
        app_name=APP_NAME,
        notebook_name=NOTEBOOK_NAME,
        notebook_namespace=NOTEBOOK_NAMESPACE,
        s3_access_key=S3_CONFIG['access_key'],
        s3_secret_key=S3_CONFIG['secret_key'],
        s3_endpoint=S3_CONFIG['endpoint']
        # app_name은 기본값을 사용하므로 전달하지 않아도 됩니다.
    )   
    
    # 데이터 처리 실행
    subway_info_overwrite(spark, S3_CONFIG, PG_CONFIG)
    

    # # SparkSession 종료
    # spark.stop()
    # print("Spark session stopped.")


 bucket name is paasup

--- Starting processing for subway_info ---
Reading CSV from s3a://paasup/datasource/서울교통공사_노선별 지하철역 정보.csv...
+----------+--------+-------------+------+--------+-------------+-------------+
|전철역코드|전철역명|전철명명_영문|  호선|외부코드|전철명명_중문|전철명명_일문|
+----------+--------+-------------+------+--------+-------------+-------------+
|      1724|    평택|   Pyeongtaek|01호선|    P165|          平?|   ピョンテク|
|      1701|    구로|         Guro|01호선|     141|         九老|         クロ|
|      1002|    남영|     Namyeong|01호선|     134|          南?|     ナミョン|
|      1706|    안양|       Anyang|01호선|    P147|          安?|     アニヤン|
|       159|  동묘앞|      Dongmyo|01호선|     127|           ??| トンミョアプ|
+----------+--------+-------------+------+--------+-------------+-------------+
only showing top 5 rows

CSV  loaded. Count: 796


25/05/28 09:46:14 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/05/28 09:46:26 WARN AmazonHttpClient: SSL Certificate checking for endpoints has been explicitly disabled.


Successfully wrote data to Delta table s3a://paasup/deltalake/subway_info.
Archiving S3 CSV file...
Copying S3 object from datasource/서울교통공사_노선별 지하철역 정보.csv to archive/서울교통공사_노선별 지하철역 정보.csv...
Deleting source S3 object: datasource/서울교통공사_노선별 지하철역 정보.csv...
Successfully moved datasource/서울교통공사_노선별 지하철역 정보.csv to archive/서울교통공사_노선별 지하철역 정보.csv.
Inserting data  into PostgreSQL table: subway_info

                        select  `전철역명` station_name
                           ,`전철역코드` station_cd
                           ,`전철명명_영문`  station_ename
                           ,`전철명명_중문`  station_cname
                           ,`전철명명_일문`  station_jname
                           ,`외부코드`  station_cd_external
                           , case when substring(`호선`,1,1) = '0' then substring(`호선`,2,length(`호선`)-1) 
                                  when `호선` = '공항철도' then '공항철도 1호선' 
                                  when `호선` = '수인분당선' then '수인선'
                                  when `호선` = '신분

/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.50.29'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '172.16.50.29'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Successfully wrote data to PostgreSQL table subway_info.
--- Processing for   completed successfully! ---


In [22]:
spark.stop()